In [3]:
# ==========================================================
#  制約内蔵型 Growing-Tree ─ 1 迷路生成＋改良表示
# ==========================================================
# パラメータはここだけ
SIZE             = 11   # 盤面一辺
MAX_STRAIGHT     = 4    # 直線通路長上限
MAX_BRANCH_DIST  = 6    # 分岐間距離上限
LOOP_RATE        = 0.15 # ループ追加確率
SEED             = None # 例: 123 で固定、None で毎回ランダム
# ----------------------------------------------------------

import random, json
from dataclasses import dataclass
from typing import List, Tuple, Dict

Cell = Tuple[int, int]

@dataclass
class Maze:
    v: List[List[bool]]   # 垂直壁 True=壁あり  (y,x) between (x,y)-(x+1,y)
    h: List[List[bool]]   # 水平壁 True=壁あり  (y,x) between (x,y)-(x,y+1)

# ---------- ヘルパー ----------
def neigh(x:int,y:int):
    if x>0:        yield 'L',x-1,y
    if x<SIZE-1:   yield 'R',x+1,y
    if y>0:        yield 'U',x,y-1
    if y<SIZE-1:   yield 'D',x,y+1

def carve(m:Maze,x:int,y:int,d:str):
    if d=='L': m.v[y][x-1]=False
    elif d=='R': m.v[y][x]  =False
    elif d=='U': m.h[y-1][x]=False
    else:        m.h[y][x]  =False   # 'D'

def wall(m:Maze,x:int,y:int,d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]

def open_square_after(m:Maze,x:int,y:int,d:str)->bool:
    blocks=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): blocks.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): blocks.append((x+dx,y+dy))
    for cx,cy in blocks:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx]; v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx]; h1=not m.h[cy][cx+1]
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

def run_len_after(m:Maze,x:int,y:int,d:str)->int:
    def run_h(px,py):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px,py):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else:        cells.append((x,y-1))
    return max(max(run_h(cx,cy),run_v(cx,cy)) for cx,cy in cells)

# ---------- ループ追加 ----------
def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE:          continue
        if open_square_after(m,x,y,d):         continue
        if run_len_after(m,x,y,d)>MAX_STRAIGHT:continue
        carve(m,x,y,d)

# ---------- Growing-Tree 生成 ----------
def generate_maze()->Maze:
    if SEED is not None:
        random.seed(SEED)

    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    info: Dict[Cell, Tuple[str,int,int,int]] = {}

    start=(random.randrange(SIZE),random.randrange(SIZE))
    frontier=[start]
    info[start]=('',0,0,0)  # prev_dir, run, dist, degree

    while frontier:
        i=random.randrange(len(frontier))
        x,y = frontier[i]
        pd, run, dist, deg = info[(x,y)]

        options=[]
        for d,nx,ny in neigh(x,y):
            if (nx,ny) in info:                          continue
            if d==pd and run>=MAX_STRAIGHT:              continue
            if dist>=MAX_BRANCH_DIST:
                # 要分岐：残り未訪問が 2 以上?
                if len([1 for t in neigh(x,y) if (t[1],t[2]) not in info])<2:
                    continue
            if open_square_after(Maze(v,h),x,y,d):       continue
            if run_len_after(Maze(v,h),x,y,d)>MAX_STRAIGHT: continue
            options.append((d,nx,ny))

        if not options:
            frontier.pop(i)
            continue

        d,nx,ny=random.choice(options)
        carve(Maze(v,h),x,y,d)
        deg+=1
        info[(x,y)]=(pd,run,dist,deg)

        new_run = run+1 if d==pd else 1
        new_dist=0 if deg>=3 else dist+1
        info[(nx,ny)]=(d,new_run,new_dist,1)
        frontier.append((nx,ny))

        # 分岐必須時に 2 本目を掘る
        if dist>=MAX_BRANCH_DIST and len(options)>=2:
            d2,nx2,ny2=random.choice([op for op in options if op!=(d,nx,ny)])
            carve(Maze(v,h),x,y,d2)
            info[(x,y)]=(pd,run,dist,deg+1)
            info[(nx2,ny2)]=(d2,1,0,1)
            frontier.append((nx2,ny2))

    maze=Maze(v,h)
    add_loops(maze)
    return maze

# ---------- 改良レンダラ (2文字幅セル) ----------
def render_pretty(m:Maze)->str:
    # 上端
    lines=[' ' + '_ '*SIZE]  # 例: " _ _ _ _ …"
    for y in range(SIZE):
        row='|'
        for x in range(SIZE):
            # 床
            floor='_' if y==SIZE-1 or m.h[y][x] else ' '
            # 右壁
            right='|' if x==SIZE-1 or m.v[y][x] else ' '
            row += floor + right
        lines.append(row)
    return '\n'.join(lines)

# ---------- 実行 ----------
maze = generate_maze()
print(render_pretty(maze))      # ← 可視化
maze_json = {                   # 必要なら JSON も取得
    "size": SIZE,
    "max_straight": MAX_STRAIGHT,
    "max_branch_dist": MAX_BRANCH_DIST,
    "loop_rate": LOOP_RATE,
    "v_walls": [[x,y] for y in range(SIZE)   for x in range(SIZE-1) if maze.v[y][x]],
    "h_walls": [[x,y] for y in range(SIZE-1) for x in range(SIZE)   if maze.h[y][x]]
}

# JSON 出力を確認したい場合は:
# import pprint; pprint.pp(maze_json)


 _ _ _ _ _ _ _ _ _ _ _ 
|_ _  |_   _  |_  | | |
| |_  | | | |   | | | |
|  _ _  | |   |  _ _ _|
|_| |  _|_  |  _   _ _|
| | |_ _  |_  | |_ _|_|
| | | |_  |  _  | | |_|
| | |_    | | | | | | |
|  _  | |_   _  | | | |
|_|_ _ _ _  |_ _  | | |
|_|_|_ _   _  |  _ _  |
|_ _ _ _ _|_ _ _ _ _|_|


In [10]:
# ==============================================================
#   Growing-Tree Maze  (no isolated cell / no 2×2 / run-len ≤ N)
# ==============================================================

SIZE             = 10      # 盤面一辺
MAX_STRAIGHT     = 7       # <- この長さを厳守
MAX_BRANCH_DIST  = 10
LOOP_RATE        = 0.25
SEED             = None    # 例: 123 で固定、None で毎回ランダム

# --------------------------------------------------------------
import random
from dataclasses import dataclass
from typing import List, Tuple, Dict

Cell = Tuple[int, int]

@dataclass
class Maze:
    v: List[List[bool]]          # True = vertical wall between (x,y)-(x+1,y)
    h: List[List[bool]]          # True = horizontal wall between (x,y)-(x,y+1)

# ---------- 基本ユーティリティ ----------
def neigh(x:int,y:int):
    if x>0:      yield 'L',x-1,y
    if x<SIZE-1: yield 'R',x+1,y
    if y>0:      yield 'U',x,y-1
    if y<SIZE-1: yield 'D',x,y+1

def carve(m:Maze,x:int,y:int,d:str):
    if d=='L': m.v[y][x-1]=False
    elif d=='R': m.v[y][x]  =False
    elif d=='U': m.h[y-1][x]=False
    else:        m.h[y][x]  =False   # 'D'

def wall(m:Maze,x:int,y:int,d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]

# ---------- 2×2 通路ブロック判定 ----------
def makes_open_square(m:Maze,x:int,y:int,d:str)->bool:
    bl=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): bl.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): bl.append((x+dx,y+dy))
    for cx,cy in bl:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx]; v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx]; h1=not m.h[cy][cx+1]
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

def has_open_square(m:Maze)->bool:
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    return False

# ---------- carve 後の最大直線長 ----------
def max_run_after(m:Maze,x:int,y:int,d:str)->int:
    def run_h(px,py):
        r=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: r+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: r+=1; ix+=1
        return r
    def run_v(px,py):
        r=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: r+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: r+=1; iy+=1
        return r
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else:        cells.append((x,y-1))
    return max(max(run_h(cx,cy),run_v(cx,cy)) for cx,cy in cells)

# ---------- 完成盤面に長すぎる直線が無いか ----------
def has_long_run(m:Maze)->bool:
    # 横方向
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run = run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 縦方向
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run = run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    return False

# ---------- ループ追加 ----------
def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d): continue
        if max_run_after(m,x,y,d)>MAX_STRAIGHT: continue
        carve(m,x,y,d)

# ---------- Growing-Tree 生成 ----------
def build_maze()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    meta: Dict[Cell,Tuple[str,int,int,int]] = {}
    start=(random.randrange(SIZE),random.randrange(SIZE))
    frontier=[start]
    meta[start]=('',0,0,0)   # prev_dir, run_len, dist_from_branch, degree

    while frontier:
        i=random.randrange(len(frontier))
        x,y=frontier[i]
        pd,run,dist,deg = meta[(x,y)]

        opts=[]
        for d,nx,ny in neigh(x,y):
            if (nx,ny) in meta: continue
            if d==pd and run>=MAX_STRAIGHT: continue
            need_branch = dist>=MAX_BRANCH_DIST
            if need_branch and len([1 for t in neigh(x,y) if (t[1],t[2]) not in meta])<2:
                continue
            if makes_open_square(Maze(v,h),x,y,d): continue
            if max_run_after(Maze(v,h),x,y,d)>MAX_STRAIGHT: continue
            opts.append((d,nx,ny))

        if not opts:
            frontier.pop(i)
            continue

        d,nx,ny=random.choice(opts)
        carve(Maze(v,h),x,y,d)
        deg+=1
        meta[(x,y)]=(pd,run,dist,deg)

        new_run = run+1 if d==pd else 1
        new_dist=0 if deg>=3 else dist+1
        meta[(nx,ny)]=(d,new_run,new_dist,1)
        frontier.append((nx,ny))

        if dist>=MAX_BRANCH_DIST and len(opts)>=2:
            d2,nx2,ny2=random.choice([o for o in opts if o!=(d,nx,ny)])
            carve(Maze(v,h),x,y,d2)
            meta[(x,y)]=(pd,run,dist,deg+1)
            meta[(nx2,ny2)]=(d2,1,0,1)
            frontier.append((nx2,ny2))

    return Maze(v,h)

# ---------- 完全生成ルーチン ----------
def generate_maze()->Maze:
    if SEED is not None:
        random.seed(SEED)

    while True:
        mz=build_maze()
        add_loops(mz)
        # 安全確認：孤立セル無し && 2×2 無し && run≤MAX
        if not has_open_square(mz) and not has_long_run(mz):
            return mz

# ---------- 2-文字幅 ASCII レンダラ ----------
def render2(m:Maze)->str:
    lines=[' '+'_'*SIZE*2]
    for y in range(SIZE):
        row='|'
        for x in range(SIZE):
            floor='_' if y==SIZE-1 or m.h[y][x] else ' '
            wall_right='|' if x==SIZE-1 or m.v[y][x] else ' '
            row += floor + wall_right
        lines.append(row)
    return '\n'.join(lines)

# ---------- 実行 ----------
maze = generate_maze()
print(render2(maze))


 ____________________
| | | | |  _   _    |
|_   _  |   |  _ _|_|
|         |_ _   _  |
|_| |_| |_ _|    _| |
|  _          |     |
|   | |_|_| |   | |_|
| | |_  |   |_|  _  |
| |  _   _| |_  | | |
| | |_    |_|  _   _|
|_|_|_ _|_ _|_|_ _ _|


In [17]:
# ==============================================================
#   Growing-Tree Maze  (no isolated cell / no 2×2 / run-len ≤ N)
# ==============================================================

SIZE             = 10      # 盤面一辺
MAX_STRAIGHT     = 7       # <- この長さを厳守
MAX_BRANCH_DIST  = 10
LOOP_RATE        = 0.25
SEED             = None    # 例: 123 で固定、None で毎回ランダム

# ---------- 実行 ----------
maze = generate_maze()
print(render2(maze))

 ____________________
|      _| | |  _    |
| |_|    _ _   _| | |
|_    |_   _ _ _ _  |
|  _|  _   _   _ _|_|
| | |   |_ _|  _ _  |
|  _| |_   _|   |_  |
|    _  |    _|  _  |
| |   | |_|_ _| |_  |
|   | |  _   _ _ _  |
|_|_|_|_ _|_ _ _ _|_|


In [60]:

# ==============================================================
#   Growing-Tree Maze  (no isolated cell / no 2×2 / run-len ≤ N)
# ==============================================================

SIZE = 5  # 盤面一辺
MAX_STRAIGHT = 3  # <- この長さを厳守
MAX_BRANCH_DIST = 5
LOOP_RATE = 0.2
SEED = None  # 例: 123 で固定、None で毎回ランダム

# ---------- 実行 ----------
maze = generate_maze()
print(render2(maze))

 __________
|   |  _  |
|_|  _  |_|
|  _  |_  |
|  _|   |_|
|_ _ _|_ _|
